## Задание: Проектирование системы доставки

**Сценарий:** Мы создаем новую систему для управления заказами и доставкой в интернет-магазине. Твоя задача — спроектировать ключевые элементы схемы базы данных, используя продвинутые типы PostgreSQL.

### Задача 1: Создание `DOMAIN`

Для обеспечения целостности данных по всей системе, создай два домена:

1.  `email_address`: Должен базироваться на `TEXT` и проверять, что значение содержит символ `@`. (Для учебного примера простая проверка достаточна).
2.  `sku_code`: Код товара (SKU). Должен быть `TEXT` и соответствовать формату `PROD-` за которым следуют ровно 6 цифр. Подсказка: используйте `VALUE ~ '^PROD-\d{6}$'`

### Задача 2: Управление статусами (`ENUM`)

Создай перечисляемый тип `delivery_status` для отслеживания статуса доставки. Он должен включать следующие состояния:

  * `packaging` (комплектация)
  * `in_transit` (в пути)
  * `delivered` (доставлен)
  * `failed` (неудачная попытка)

### Задача 3: Моделирование получателя (`COMPOSITE TYPE`)

Информация о получателе — это единая сущность. Создай составной тип `recipient_info`, который будет содержать:

  * `full_name` (TEXT)
  * `contact_email` (используй созданный домен `email_address`)
  * `postal_code` (TEXT, 5 цифр)

### Задача 4: Проектирование основной таблицы и работа с `JSONB`

Теперь соберем все вместе.

1.  Создай таблицу `deliveries` со следующими полями:

      * `id`: `SERIAL PRIMARY KEY`
      * `order_number`: `INTEGER`
      * `item_sku`: `sku_code` (используй домен)
      * `status`: `delivery_status` (используй ENUM)
      * `recipient`: `recipient_info` (используй составной тип)
      * `metadata`: `JSONB` (для хранения дополнительной информации, например, вес посылки, оценочная стоимость, комментарии курьера).

2.  **Заполни таблицу данными:** Вставь 3-4 тестовые доставки. Убедись, что:

      * Одна из доставок имеет статус `delivered`.
      * Одна запись в `metadata` содержит ключ `"is_fragile": true` (хрупкий товар).
      * Одна запись в `metadata` содержит ключ `"courier_comment"` с каким-нибудь текстом.
      * Данные для `recipient` вставляются корректно.

3.  **Напиши запросы:**

      * Выведи имена всех получателей (`full_name` из `recipient`) и их почтовые индексы для доставок, а также хотя бы 1 любое значение из `metadata`, для заказов с состоянием `in_transit`.
      * Найди все доставки хрупких товаров (`is_fragile` = `true`).
      * Посчитай количество доставок для каждого статуса.

### Задача 5: Эволюция схемы (`ALTER TYPE`)

Бизнес не стоит на месте, и требования изменились.

1.  В `delivery_status` нужно добавить новый статус `returning` (возвращается на склад) после статуса `failed`.
2.  В составной тип `recipient_info` нужно добавить поле `phone_number` (VARCHAR), затем попытайтесь изменить тип на TEXT (получите ошибку).
3.  На домен `sku_code` нужно добавить еще одно ограничение с именем `non_zero_check`, которое проверяет, что числовая часть кода не равна `000000`. Для этого используйте: `CHECK (SUBSTRING(VALUE FROM 6)::INT != 0)`
4.  В `metadata` к существующим полям добавьте новый ключ 

### Задача 6: Оптимизация (`INDEX`)

Аналитики часто ищут доставки по почтовому индексу получателя и по наличию комментариев от курьера.

1.  Создай индекс для быстрого поиска по полю `postal_code` внутри составного типа `recipient`.
2.  Создай GIN-индекс на колонку `metadata` для ускорения поиска по ее ключам.